## In this notebook, we will process and clean the raw data that is provided to train the model.

* To begine with, first lets load the data and display the first few lines to know what we are working with.

In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [75]:
file_path = 'C:/Users/ok/Desktop/kaggel_titanic_project/Kaggle_titanic/Data/train.csv'
file_path1 = 'C:/Users/ok/Desktop/kaggel_titanic_project/Kaggle_titanic/Data/test.csv'
data = pd.read_csv(file_path, index_col='PassengerId')
test = pd.read_csv(file_path1)
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [26]:
data.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


* From the above cell, we  can see that most of the columons have data in them (are not empty). But we can see that the cabin has the most Nan values(more than half). And the age columon also has the second most highest Nan values.

* Hence since the cabin columon has more than 20% NaN value, i have decided to drop the cabin columon from the data, and impend the value for the age Nan values. And alos to for the two rows which has aNan value for the Embark columon

In [32]:
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [33]:
#now lets split the usable data into feature (X) and prediction (y), and split the data into training and validation
y_train = data['Survived']
#data.drop(['Survived'], axis=1, inplace=True)
X_train = data.drop(['Survived'], axis= 1)
#Before training the model with the training data lets calculate how accurate it is. 


In [42]:
#First lets split the data into numerical data and catagorical data(data with 'object')
num_col = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]
obj_col = [col for col in X_train.columns if X_train[col].dtype in ['object']]

In [43]:
#Note that we need to convert the object data into numerical value for training the mcahine.
#Hence we need to use a good data (data with low cardinality)
cat_col = [cols for cols in obj_col if X_train[cols].nunique()< 10]
data_f = data[cat_col + num_col]
cols = cat_col + num_col

In [45]:
X_train = X_train[cols]
test = test[cols]

In [46]:
#now lets preprocess the numical_datas ans the catagorical data by transformers
numerical_transformer = SimpleImputer(strategy='mean')
catagorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('numerical', numerical_transformer, num_col),
    ('catagorical', catagorical_transformer, cat_col)
])

In [47]:
#now definr the model
model = RandomForestRegressor (random_state=0)

In [51]:
#finally we can define the pipeline
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('mode', model)
])

In [52]:
#now lets train the model over the train X ans y data then predict the valid X data
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical', SimpleImputer(),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('catagorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked'])])),
                ('mode', RandomForestRegressor(random_state=0))])

In [54]:
prediction = pipe.predict(test)

In [80]:
# Apply thresholding
prediction = (prediction >= 0.5).astype(int)
# Save the updated submission file
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': prediction})
submission.to_csv('submissions.csv', index=False)
